In [8]:
import numpy as np
import pandas as pd
import json
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
destinations = pd.read_csv("Destinations.csv")

In [9]:
with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
with open("label_encoders.pkl", "rb") as f:
    label_encoders = pickle.load(f)
with open("destination_mapping.json", "r") as f:
    destination_mapping = json.load(f)

In [10]:
reverse_mapping = {v: k for k, v in destination_mapping.items()}

In [11]:
df = pd.read_csv("Destinations.csv")

In [12]:
def preprocess_input(user_input):
    categorical_cols = ["Climate Type", "Best Visiting Season", "Budget Level", "Crime Index"]
    numeric_cols = ["User Ratings (out of 5)", "Social Media Mentions", "Number of Reviews"]

    processed_input = {}

    # Encode categorical features
    for col in categorical_cols:
        if user_input[col] in label_encoders[col].classes_:
            processed_input[col] = label_encoders[col].transform([user_input[col]])[0]
        else:
            print(f"Warning: '{user_input[col]}' not in training data for {col}. Assigning default category.")
            processed_input[col] = 0  # Assign default value

    # Convert to DataFrame
    df_input = pd.DataFrame([processed_input])

    # Process numeric values
    df_input[numeric_cols] = pd.DataFrame([user_input], index=[0])[numeric_cols]

    # Scale numeric features
    df_input[numeric_cols] = scaler.transform(df_input[numeric_cols])

    return df_input.values.astype(np.float32)  # Ensure float32 type

In [13]:
def get_recommendations(user_input, top_n=5):
    X_new = preprocess_input(user_input)

    # Extract destination feature vectors (assuming precomputed TF-IDF or other embeddings exist)
    destination_features = df.drop(columns=["_id"]).values

    # Compute cosine similarity between user input and all destinations
    similarities = cosine_similarity(X_new, destination_features)[0]

    # Get top-N recommendations
    top_indices = similarities.argsort()[::-1][:top_n]

    # Return destination names
    return [reverse_mapping[df.iloc[i]["_id"]] for i in top_indices]


In [14]:
user_input = {
    "Climate Type": "Tropical",
    "Best Visiting Season": "Winter",
    "Budget Level": "Medium",
    "Crime Index": "Low",
    "User Ratings (out of 5)": 4.5,
    "Social Media Mentions": 100000,
    "Number of Reviews": 2000
}

In [15]:
recommendations = get_recommendations(user_input, top_n=5)
print("Recommended Destinations:", recommendations)

ValueError: could not convert string to float: 'Cambridge'